## San Antonio Riverwalk Hotel
8760 rows per simulation<br>

## Preprocess

1. Generate zone with facade glazing systems in COMFEN.

2. Retrieve IDFs for each scenario from the COMFEN5 Results folder.Then IDF files are copied after each simulation. These files will be used in the Jupyter Notebook.

3. Convert from E+ 8.1 to 9.5 using the .lst format.

4. Use E+ template file (PNNL large hotel sample file) to change the outputs. See Word doc that describes the process. Process all IDFs. Window data already in each IDF, so no need to cut and paste.

5. Rerun batch sims in EnergyPlus v9.5.

6. Add "Scenario", "Elevation, and "GlazingSystem" to the results CSV files manually before importing into this notebook.

In [1]:
# Dependencies
import pandas as pd
import os
import numpy as np
from pathlib import Path
from datetime import datetime

## Energy Conversions

As per E* Documentation<br>

Joules --> kBtu (multiply by .00000094781712)<br>
Joules --> kWh (multiply by .000000277777778) or (divide by 3600000)<br>
Joules --> therms (multiply by .0000000094781712<br>

<div>
    <img src="static/images/elect_conv.png" width="400"/>
     <img src="static/images/gas_conv.png" width="400"/>
</div

## Energy Costs and Emission Factors

http://seeatcalc.gastechnology.org/CommercialBuildings

In [2]:
# Costs (elec $/kWh, gas $/therm), change values per city
# State Average
# comgas_cost = 0.95
# comelec_cost = 0.1743

# Provided by client
# Averaged off-peak costs -- largest percent of usage
comgas_cost = 0.0  #left at 0 so gas is not counted 0.64 $/therm)
comelec_cost = 0.076

In [3]:
# Source Energy Conversion Factors (same for res or comm), change values per city
gas_source = 1.09
elec_source = 2.53

In [4]:
# Carbon equivalent factors (same for res or comm), change values per city

#Electricity (lb/MMBtu) 323.34 (lb/kbtu).32334 ####
#Natural Gas (Building Used, lb/MMBtu) 145.66 (lb/kBtu .14566) ####

gas_co2 = 0.14566
elec_co2 = 0.11033

In [5]:
# Zone size for EUI and cost/sf

sf = 187.5

In [6]:
# Lighting energy reduction, assumption that LEDs are being used. 
# Use this multiplier if reductions are not in the COMFEN model

lighting_redux = 1

In [7]:
# # For peak demand calculations (.0002931 kW = 1 Btu).

# kw = 0.0002931

---

## Combine CSV Files in Directory 

In [8]:
# Create variable for files in directory
files = [f for f in os.listdir("data/sat-riverwalk/") if f.endswith(".csv")]

# files

In [9]:
# https://stackoverflow.com/questions/63886787/how-to-create-a-dataframe-from-multiple-csv-files
# Initialize and empty dataframe
all_ComElec = pd.DataFrame()

# Iterate through files and contents, then concatenate the data into the dataframe
for file in files:
    df = pd.read_csv("data/sat-riverwalk/" + file)
    all_ComElec = pd.concat([all_ComElec, df])
    
all_ComElec

# 52560 rows × 46 columns

,Scenario,Elevation,GlazingSystem,Date/Time,Environment:Site Outdoor Air Drybulb Temperature [C](Hourly),COMFENZONE:Zone Lights Electricity Energy [J](Hourly),COMFENZONE:Zone Mean Radiant Temperature [C](Hourly),COMFENZONE:Zone Mean Air Temperature [C](Hourly),COMFENZONE:Zone Operative Temperature [C](Hourly),COMFENZONE:Zone Air System Sensible Heating Rate [W](Hourly),...,COMFENZONE:Zone Thermal Comfort ASHRAE 55 Simple Model Summer or Winter Clothes Not Comfortable Time [hr](Hourly),COMFENZONE:Zone Heating Setpoint Not Met While Occupied Time [hr](Hourly),COMFENZONE:Zone Cooling Setpoint Not Met While Occupied Time [hr](Hourly),SYS-1 DX COOLING COIL:Cooling Coil Total Cooling Rate [W](Hourly),SYS-1 DX COOLING COIL:Cooling Coil Total Cooling Energy [J](Hourly),SYS-1 DX COOLING COIL:Cooling Coil Sensible Cooling Rate [W](Hourly),SYS-1 DX COOLING COIL:Cooling Coil Sensible Cooling Energy [J](Hourly),SYS-1 HEATING COIL:Heating Coil Heating Energy [J](Hourly),SYS-1 HEATING COIL:Heating Coil Heating Rate [W](Hourly),SYS-1:Air System Fan Electricity Energy [J](Hourly)
0,e-singletint-clear,east,clear panel,01/01 01:00:00,7.8375,35099.99643,23.235360,23.221029,23.228195,0.000000,...,0.0,0.0,0.0,0.000000,0.00000,0.000000,0.00000,0.00000,0.000000,376064.5314
1,e-singletint-clear,east,clear panel,01/01 02:00:00,6.8875,35099.99643,22.932457,22.861256,22.896857,0.000000,...,0.0,0.0,0.0,0.000000,0.00000,0.000000,0.00000,0.00000,0.000000,376064.5314
2,e-singletint-clear,east,clear panel,01/01 03:00:00,6.0125,14849.99849,22.628975,22.525268,22.577122,0.000000,...,0.0,0.0,0.0,0.000000,0.00000,0.000000,0.00000,0.00000,0.000000,376064.5314
3,e-singletint-clear,east,clear panel,01/01 04:00:00,5.9125,14849.99849,22.351630,22.234302,22.292966,0.000000,...,0.0,0.0,0.0,0.000000,0.00000,0.000000,0.00000,0.00000,0.000000,376064.5314
4,e-singletint-clear,east,clear panel,01/01 05:00:00,6.1000,14849.99849,22.125860,22.041056,22.083458,0.000000,...,0.0,0.0,0.0,0.000000,0.00000,0.000000,0.00000,0.00000,0.000000,376064.5314
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8755,w-singletint,west,existing,12/31 20:00:00,15.6375,134999.98630,24.160832,23.999942,24.080387,0.000000,...,0.0,0.0,0.0,108.554110,390794.79650,91.604471,329776.09510,0.00000,0.000000,328144.2073
8756,w-singletint,west,existing,12/31 21:00:00,13.2500,134999.98630,23.753488,23.985367,23.869427,0.178985,...,0.0,0.0,0.0,23.205321,83539.15572,18.734348,67443.65347,0.00000,0.000000,328144.2073
8757,w-singletint,west,existing,12/31 22:00:00,11.5125,134999.98630,23.373504,23.647915,23.510709,8.156992,...,0.0,0.0,0.0,0.000000,0.00000,0.000000,0.00000,23793.38049,6.609272,328144.2073
8758,w-singletint,west,existing,12/31 23:00:00,10.7875,114749.98830,22.931725,23.022069,22.976897,19.908872,...,0.0,0.0,0.0,0.000000,0.00000,0.000000,0.00000,48645.53830,13.512650,328144.2073


In [10]:
list(all_ComElec.columns.values)

['Scenario',
 'Elevation',
 'GlazingSystem',
 'Date/Time',
 'Environment:Site Outdoor Air Drybulb Temperature [C](Hourly)',
 'COMFENZONE:Zone Lights Electricity Energy [J](Hourly)',
 'COMFENZONE:Zone Mean Radiant Temperature [C](Hourly)',
 'COMFENZONE:Zone Mean Air Temperature [C](Hourly)',
 'COMFENZONE:Zone Operative Temperature [C](Hourly)',
 'COMFENZONE:Zone Air System Sensible Heating Rate [W](Hourly)',
 'COMFENZONE:Zone Air System Sensible Cooling Rate [W](Hourly)',
 'COMFENZONE:Zone Air Temperature [C](Hourly)',
 'COMFENZONE:Zone Air Humidity Ratio [](Hourly)',
 'COMFENZONE:Zone Air Relative Humidity [%](Hourly)',
 'COMFENZONE:Zone Thermal Comfort ASHRAE 55 Simple Model Summer or Winter Clothes Not Comfortable Time [hr](Hourly)',
 'COMFENZONE:Zone Heating Setpoint Not Met While Occupied Time [hr](Hourly)',
 'COMFENZONE:Zone Cooling Setpoint Not Met While Occupied Time [hr](Hourly)',
 'SYS-1 DX COOLING COIL:Cooling Coil Total Cooling Rate [W](Hourly)',
 'SYS-1 DX COOLING COIL:Cool

In [11]:
# Get wanted columns
all_energy = all_ComElec[["Scenario",
               "Date/Time",
                "Elevation",
               "GlazingSystem",
              "COMFENZONE:Zone Lights Electricity Energy [J](Hourly)",
              "SYS-1 DX COOLING COIL:Cooling Coil Total Cooling Energy [J](Hourly)",
              "SYS-1 HEATING COIL:Heating Coil Heating Energy [J](Hourly)",
             "SYS-1:Air System Fan Electricity Energy [J](Hourly) "]]

# all_energy

In [12]:
all_energy = all_energy.dropna().reset_index(drop=True)

all_energy

,Scenario,Date/Time,Elevation,GlazingSystem,COMFENZONE:Zone Lights Electricity Energy [J](Hourly),SYS-1 DX COOLING COIL:Cooling Coil Total Cooling Energy [J](Hourly),SYS-1 HEATING COIL:Heating Coil Heating Energy [J](Hourly),SYS-1:Air System Fan Electricity Energy [J](Hourly)
0,e-singletint-clear,01/01 01:00:00,east,clear panel,35099.99643,0.00000,0.00000,376064.5314
1,e-singletint-clear,01/01 02:00:00,east,clear panel,35099.99643,0.00000,0.00000,376064.5314
2,e-singletint-clear,01/01 03:00:00,east,clear panel,14849.99849,0.00000,0.00000,376064.5314
3,e-singletint-clear,01/01 04:00:00,east,clear panel,14849.99849,0.00000,0.00000,376064.5314
4,e-singletint-clear,01/01 05:00:00,east,clear panel,14849.99849,0.00000,0.00000,376064.5314
...,...,...,...,...,...,...,...,...
105115,w-singletint,12/31 20:00:00,west,existing,134999.98630,390794.79650,0.00000,328144.2073
105116,w-singletint,12/31 21:00:00,west,existing,134999.98630,83539.15572,0.00000,328144.2073
105117,w-singletint,12/31 22:00:00,west,existing,134999.98630,0.00000,23793.38049,328144.2073
105118,w-singletint,12/31 23:00:00,west,existing,114749.98830,0.00000,48645.53830,328144.2073


In [13]:
all_energy_group = all_energy.groupby("Scenario")

# all_energy_group.sum()

In [14]:
# Export grouped file to csv
all_energy.to_csv("Scraper_Output/riverwalk_all.csv", header=True, index=True)

In [15]:
# # Export grouped file to csv
# all_energy_group.to_csv("Scraper_Output/msp2_group.csv", header=True, index=True)

-----

## Get the Wanted Data (columns)

In [16]:
# Rename Columns
all_energy2 = all_energy.rename(columns={"Scenario":"Scenario",
                              "Date/Time":"Date_Time",
                             "SYS-1:Air System Fan Electricity Energy [J](Hourly) ":"FanEnergy[J](Hourly)",
                             "SYS-1 DX COOLING COIL:Cooling Coil Total Cooling Energy [J](Hourly)":"CoolingEnergy[J](Hourly)",
                            "SYS-1 HEATING COIL:Heating Coil Heating Energy [J](Hourly)":"HeatingEnergy[J](Hourly)",
                            "COMFENZONE:Zone Lights Electricity Energy [J](Hourly)":"LightingEnergy[J](Hourly)"
                             })

# all_energy2

In [17]:
list(all_energy2.columns.values)

['Scenario',
 'Date_Time',
 'Elevation',
 'GlazingSystem',
 'LightingEnergy[J](Hourly)',
 'CoolingEnergy[J](Hourly)',
 'HeatingEnergy[J](Hourly)',
 'FanEnergy[J](Hourly)']

-----

## Get Annual Energy Data
Convert units from Joules to kBtu<br>

In [18]:
# Copy dataframe to modify and leave original df intact
get_annual = all_energy2.copy(deep=True)

# get_annual

In [19]:
# list(get_annual.columns.values)

In [20]:
# Convert Joules to KBtu in dataframe
get_annual["FanEnergy(kBtu)"] = get_annual["FanEnergy[J](Hourly)"] * .00000094781712
get_annual["HeatingEnergy(kBtu)"] = get_annual["HeatingEnergy[J](Hourly)"] * .00000094781712
get_annual["CoolingEnergy(kBtu)"] = get_annual["CoolingEnergy[J](Hourly)"] * .00000094781712

# Reduce lighting energy, assuming LEDs are being used
get_annual["LightingEnergy(kBtu)"] = (get_annual["LightingEnergy[J](Hourly)"] * .00000094781712) * lighting_redux

# get_annual

In [21]:
# Drop columns
get_annual.drop(columns = ["FanEnergy[J](Hourly)", "HeatingEnergy[J](Hourly)",  "CoolingEnergy[J](Hourly)", "LightingEnergy[J](Hourly)"], axis=1, inplace=True)

# get_annual

In [22]:
# Add energy totals
totalEnergy = get_annual["FanEnergy(kBtu)"] + get_annual["HeatingEnergy(kBtu)"] + get_annual["LightingEnergy(kBtu)"] + get_annual["CoolingEnergy(kBtu)"]
        
get_annual["AnnualEnergy(kBtu)"] = totalEnergy

get_annual

,Scenario,Date_Time,Elevation,GlazingSystem,FanEnergy(kBtu),HeatingEnergy(kBtu),CoolingEnergy(kBtu),LightingEnergy(kBtu),AnnualEnergy(kBtu)
0,e-singletint-clear,01/01 01:00:00,east,clear panel,0.356440,0.000000,0.000000,0.033268,0.389709
1,e-singletint-clear,01/01 02:00:00,east,clear panel,0.356440,0.000000,0.000000,0.033268,0.389709
2,e-singletint-clear,01/01 03:00:00,east,clear panel,0.356440,0.000000,0.000000,0.014075,0.370515
3,e-singletint-clear,01/01 04:00:00,east,clear panel,0.356440,0.000000,0.000000,0.014075,0.370515
4,e-singletint-clear,01/01 05:00:00,east,clear panel,0.356440,0.000000,0.000000,0.014075,0.370515
...,...,...,...,...,...,...,...,...,...
105115,w-singletint,12/31 20:00:00,west,existing,0.311021,0.000000,0.370402,0.127955,0.809378
105116,w-singletint,12/31 21:00:00,west,existing,0.311021,0.000000,0.079180,0.127955,0.518156
105117,w-singletint,12/31 22:00:00,west,existing,0.311021,0.022552,0.000000,0.127955,0.461528
105118,w-singletint,12/31 23:00:00,west,existing,0.311021,0.046107,0.000000,0.108762,0.465890


-----

## Get Energy per sf 
Add EUI (site) column<br>


In [23]:
# Get data per sf (energy/sf)
get_annual["FanEnergy(kBtu/sf)"] = (get_annual["FanEnergy(kBtu)"]) / sf
get_annual["HeatingEnergy(kBtu/sf)"] = (get_annual["HeatingEnergy(kBtu)"]) / sf
get_annual["CoolingEnergy(kBtu/sf)"] = (get_annual["CoolingEnergy(kBtu)"]) / sf
get_annual["LightingEnergy(kBtu/sf)"] = (get_annual["LightingEnergy(kBtu)"]) / sf

In [24]:
# Establish EUI (energy/sf)
get_annual["EUI(kBtu/sf)"] = (get_annual["FanEnergy(kBtu)"] +
                                    get_annual["HeatingEnergy(kBtu)"] +
                                    get_annual["LightingEnergy(kBtu)"] +  
                                    get_annual["CoolingEnergy(kBtu)"]) / sf
                                                       
# get_annual

-----

## Get Carbon per sf 
Add carbon/sf column

In [25]:
# Do carbon calculation using variable (lb/sf)
get_annual["CO2_Electric(lb/sf)"] = ((get_annual["FanEnergy(kBtu)"] +
                                    get_annual["LightingEnergy(kBtu)"] +
                                    get_annual["HeatingEnergy(kBtu)"]+
                                    get_annual["CoolingEnergy(kBtu)"]) * elec_co2) / sf
                                                       
get_annual

,Scenario,Date_Time,Elevation,GlazingSystem,FanEnergy(kBtu),HeatingEnergy(kBtu),CoolingEnergy(kBtu),LightingEnergy(kBtu),AnnualEnergy(kBtu),FanEnergy(kBtu/sf),HeatingEnergy(kBtu/sf),CoolingEnergy(kBtu/sf),LightingEnergy(kBtu/sf),EUI(kBtu/sf),CO2_Electric(lb/sf)
0,e-singletint-clear,01/01 01:00:00,east,clear panel,0.356440,0.000000,0.000000,0.033268,0.389709,0.001901,0.000000,0.000000,0.000177,0.002078,0.000229
1,e-singletint-clear,01/01 02:00:00,east,clear panel,0.356440,0.000000,0.000000,0.033268,0.389709,0.001901,0.000000,0.000000,0.000177,0.002078,0.000229
2,e-singletint-clear,01/01 03:00:00,east,clear panel,0.356440,0.000000,0.000000,0.014075,0.370515,0.001901,0.000000,0.000000,0.000075,0.001976,0.000218
3,e-singletint-clear,01/01 04:00:00,east,clear panel,0.356440,0.000000,0.000000,0.014075,0.370515,0.001901,0.000000,0.000000,0.000075,0.001976,0.000218
4,e-singletint-clear,01/01 05:00:00,east,clear panel,0.356440,0.000000,0.000000,0.014075,0.370515,0.001901,0.000000,0.000000,0.000075,0.001976,0.000218
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
105115,w-singletint,12/31 20:00:00,west,existing,0.311021,0.000000,0.370402,0.127955,0.809378,0.001659,0.000000,0.001975,0.000682,0.004317,0.000476
105116,w-singletint,12/31 21:00:00,west,existing,0.311021,0.000000,0.079180,0.127955,0.518156,0.001659,0.000000,0.000422,0.000682,0.002763,0.000305
105117,w-singletint,12/31 22:00:00,west,existing,0.311021,0.022552,0.000000,0.127955,0.461528,0.001659,0.000120,0.000000,0.000682,0.002461,0.000272
105118,w-singletint,12/31 23:00:00,west,existing,0.311021,0.046107,0.000000,0.108762,0.465890,0.001659,0.000246,0.000000,0.000580,0.002485,0.000274


In [26]:
# get_annual["CO2_Gas(lb/sf)"] = ((get_annual["HeatingEnergy(kBtu)"]) * gas_co2) / sf
                                                       
# # get_annual

## Get Annual Cost Data
Convert units from Joules to kWh and therms to use cost multiplier<br>
Add cost multiplier, variables defined above

In [27]:
# Get annual costs
get_annual_costs = all_energy2.copy(deep=True)

In [28]:
# Convert Joules to KBtu in dataframe and add cost multiplier
# Gas kbtu => Mbtu => therms: (kbtu/1000) * 10
# Elec kbtu => Kwh (kbtu * .000000277777778)
get_annual_costs["FanCost($)"] = (get_annual_costs["FanEnergy[J](Hourly)"] * .000000277777778) * comelec_cost
# get_annual_costs["HeatingCost($)"] = ((get_annual_costs["HeatingEnergy[J](Hourly)"] * .00000094781712)/1000) * 10  * comgas_cost
get_annual_costs["HeatingCost($)"] = (get_annual_costs["HeatingEnergy[J](Hourly)"] * .000000277777778) * comelec_cost
get_annual_costs["CoolingCost($)"] = (get_annual_costs["CoolingEnergy[J](Hourly)"] * .000000277777778) * comelec_cost

# Reduce lighting energy, assuming LEDs are being used
get_annual_costs["LightingCost($)"] = (get_annual_costs["LightingEnergy[J](Hourly)"] * .000000277777778) * lighting_redux * comelec_cost

#get_annual_costs

In [29]:
# Drop columns
get_annual_costs.drop(columns = ["FanEnergy[J](Hourly)", "HeatingEnergy[J](Hourly)", "CoolingEnergy[J](Hourly)", "LightingEnergy[J](Hourly)"], axis=1, inplace=True)

#get_annual_costs

In [30]:
# Add cost totals
totalCost = get_annual_costs["FanCost($)"] +  get_annual_costs["HeatingCost($)"]  + get_annual_costs["CoolingCost($)"] + get_annual_costs["LightingCost($)"]
        
get_annual_costs["AnnualCost($)"] = totalCost

get_annual_costs

,Scenario,Date_Time,Elevation,GlazingSystem,FanCost($),HeatingCost($),CoolingCost($),LightingCost($),AnnualCost($)
0,e-singletint-clear,01/01 01:00:00,east,clear panel,0.007939,0.000000,0.000000,0.000741,0.008680
1,e-singletint-clear,01/01 02:00:00,east,clear panel,0.007939,0.000000,0.000000,0.000741,0.008680
2,e-singletint-clear,01/01 03:00:00,east,clear panel,0.007939,0.000000,0.000000,0.000313,0.008253
3,e-singletint-clear,01/01 04:00:00,east,clear panel,0.007939,0.000000,0.000000,0.000313,0.008253
4,e-singletint-clear,01/01 05:00:00,east,clear panel,0.007939,0.000000,0.000000,0.000313,0.008253
...,...,...,...,...,...,...,...,...,...
105115,w-singletint,12/31 20:00:00,west,existing,0.006927,0.000000,0.008250,0.002850,0.018028
105116,w-singletint,12/31 21:00:00,west,existing,0.006927,0.000000,0.001764,0.002850,0.011541
105117,w-singletint,12/31 22:00:00,west,existing,0.006927,0.000502,0.000000,0.002850,0.010280
105118,w-singletint,12/31 23:00:00,west,existing,0.006927,0.001027,0.000000,0.002422,0.010377


In [31]:
# Need to add square footage per elevation to the export, or add it via Tableau (if equal on all elevations)
costx = get_annual_costs.groupby(["Scenario"], as_index=True)

costx.sum()

,FanCost($),HeatingCost($),CoolingCost($),LightingCost($),AnnualCost($)
Scenario,,,,,
e-singletint,64.964793,18.401142,366.124219,10.289154,459.779309
e-singletint-clear,69.546867,2.826024,344.404785,10.289154,427.066831
e-singletint-lowe,49.128772,6.349770,265.221313,10.289154,330.989009
n-singletint,19.481799,38.868245,177.990218,10.289154,246.629417
n-singletint-clear,25.004643,12.106144,201.540266,10.289154,248.940208
n-singletint-lowe,20.721089,16.172950,169.335642,10.289154,216.518836
s-singletint,169.294189,4.755136,491.221655,10.289154,675.560135
s-singletint-clear,146.325870,0.193284,441.899581,10.289154,598.707889
s-singletint-lowe,103.937748,1.415051,331.011694,10.289154,446.653648


-----

## Get Annual Cost per sf Data
Add costs/sf columns<br>


In [32]:
# Get data per sf ($/sf)
get_annual_costs["FanCost($/sf)"] = (get_annual_costs["FanCost($)"]) / sf
get_annual_costs["HeatingCost($/sf)"] = (get_annual_costs["HeatingCost($)"]) / sf
get_annual_costs["CoolingCost($/sf)"] = (get_annual_costs["CoolingCost($)"]) / sf
get_annual_costs["LightingCost($/sf)"] = (get_annual_costs["LightingCost($)"]) / sf

# get_annual_costs_ComElec

In [33]:
# Establish costs ($/sf)
get_annual_costs["AnnualCost($/sf)"] = (get_annual_costs["FanCost($)"] +
                                    get_annual_costs["HeatingCost($)"] +  
                                    get_annual_costs["CoolingCost($)"] +  
                                    get_annual_costs["LightingCost($)"]) / sf
                                                       
# get_annual_costs

---

## Combine Annual Energy and Cost Data
Combine two dfs<br>
Export combine file

In [34]:
# # Combine data for energy and cost
allAnnualData = pd.merge(get_annual, get_annual_costs)

# allAnnualData

In [35]:
# list(allAnnualDataComElec.columns.values)

---
## Group and Export Annual Data
Group dataframe by scenario<br>
Export to file

In [36]:
# Group by scenario
annualData = allAnnualData.groupby(["Scenario", "Elevation", "GlazingSystem"], as_index=True)

# annualData.sum()

In [37]:
annualDataFinal = annualData.sum()

# annualDataFinal

In [38]:
# list(annualDataFinal.columns.values)

In [39]:
# Export grouped file to csv
annualDataFinal.to_csv("Scraper_Output/riverwalk_annual.csv", header=True, index=True)

---

## Establish dataframes for each condition to determine delta from existing

Create dataset with just existing condition.<br> 
Created a dataset with clear panel. <br>
Create a dataset with lowe panel.<br>
Keep the units in kBtus to be able to eventually combine heating and cooling <br>
Export to CSV files


In [40]:
# Define the scenario for the existing condition.
baselineAnnualData = allAnnualData[allAnnualData["GlazingSystem"] == "existing"]

# baselineAnnualData

In [41]:
# Rename columns to indicate baseline to prepare for merge"

allBaselineAnnualData = baselineAnnualData.rename(columns={'Scenario':'B_Scenario',
 'Date_Time':'B_Date_Time',
 'Elevation':'B_Elevation',
 'GlazingSystem':'B_GlazingSystem',
 'FanEnergy(kBtu)':'B_FanEnergy(kBtu)',
 'HeatingEnergy(kBtu)':'B_HeatingEnergy(kBtu)',
 'CoolingEnergy(kBtu)':'B_CoolingEnergy(kBtu)',
 'LightingEnergy(kBtu)':'B_LightingEnergy(kBtu)',
 'AnnualEnergy(kBtu)':'B_AnnualEnergy(kBtu)',
 'FanEnergy(kBtu/sf)':'B_FanEnergy(kBtu/sf)',
 'HeatingEnergy(kBtu/sf)':'B_HeatingEnergy(kBtu/sf)',
 'CoolingEnergy(kBtu/sf)':'B_CoolingEnergy(kBtu/sf)',
 'LightingEnergy(kBtu/sf)':'B_LightingEnergy(kBtu/sf)',
 'EUI(kBtu/sf)':'B_EUI(kBtu/sf)',
 'CO2_Electric(lb/sf)':'B_CO2_Electric(lb/sf)',
 'CO2_Gas(lb/sf)':'B_CO2_Gas(lb/sf)',
 'FanCost($)':'B_FanCost($)',
 'HeatingCost($)':'B_HeatingCost($)',
 'CoolingCost($)':'B_CoolingCost($)',
 'LightingCost($)':'B_LightingCost($)',
 'AnnualCost($)':'B_AnnualCost($)',
 'FanCost($/sf)':'B_FanCost($/sf)',
 'HeatingCost($/sf)':'B_HeatingCost($/sf)',
 'CoolingCost($/sf)':'B_CoolingCost($/sf)',
 'LightingCost($/sf)':'B_LightingCost($/sf)',
 'AnnualCost($/sf)':'B_AnnualCost($/sf)'
                            })

# allBaselineAnnualDataComElec

In [42]:
# Reset index
allBaselineAnnualData.reset_index(inplace=True, drop=True)

# allBaselineAnnualData

In [43]:
# # Define the scenario for the baselines.

clearAnnualData = allAnnualData[allAnnualData["GlazingSystem"] == "clear panel"]

# clearAnnualData

In [44]:
# Rename columns to indicate baseline to prepare for merge"

allClearAnnualData = clearAnnualData.rename(columns={'Scenario':'C_Scenario',
 'Date_Time':'C_Date_Time',
 'Elevation':'C_Elevation',
 'GlazingSystem':'C_GlazingSystem',
 'FanEnergy(kBtu)':'C_FanEnergy(kBtu)',
 'HeatingEnergy(kBtu)':'C_HeatingEnergy(kBtu)',
 'CoolingEnergy(kBtu)':'C_CoolingEnergy(kBtu)',
 'LightingEnergy(kBtu)':'C_LightingEnergy(kBtu)',
 'AnnualEnergy(kBtu)':'C_AnnualEnergy(kBtu)',
 'FanEnergy(kBtu/sf)':'C_FanEnergy(kBtu/sf)',
 'HeatingEnergy(kBtu/sf)':'C_HeatingEnergy(kBtu/sf)',
 'CoolingEnergy(kBtu/sf)':'C_CoolingEnergy(kBtu/sf)',
 'LightingEnergy(kBtu/sf)':'C_LightingEnergy(kBtu/sf)',
 'EUI(kBtu/sf)':'C_EUI(kBtu/sf)',
 'CO2_Electric(lb/sf)':'C_CO2_Electric(lb/sf)',
 'CO2_Gas(lb/sf)':'C_CO2_Gas(lb/sf)',
 'FanCost($)':'C_FanCost($)',
 'HeatingCost($)':'C_HeatingCost($)',
 'CoolingCost($)':'C_CoolingCost($)',
 'LightingCost($)':'C_LightingCost($)',
 'AnnualCost($)':'C_AnnualCost($)',
 'FanCost($/sf)':'C_FanCost($/sf)',
 'HeatingCost($/sf)':'C_HeatingCost($/sf)',
 'CoolingCost($/sf)':'C_CoolingCost($/sf)',
 'LightingCost($/sf)':'C_LightingCost($/sf)',
 'AnnualCost($/sf)':'C_AnnualCost($/sf)'
                            })

# allClearAnnualData

In [45]:
# Reset index
allClearAnnualData.reset_index(inplace=True, drop=True)

# allClearAnnualData

In [46]:
# Define the scenario for the baselines.

loweAnnualData = allAnnualData[allAnnualData["GlazingSystem"] == "lowe panel"]

# loweAnnualData

In [47]:
# Rename columns to indicate baseline to prepare for merge"

allLoweAnnualData = loweAnnualData.rename(columns={'Scenario':'L_Scenario',
 'Date_Time':'L_Date_Time',
 'Elevation':'L_Elevation',
 'GlazingSystem':'L_GlazingSystem',
 'FanEnergy(kBtu)':'L_FanEnergy(kBtu)',
 'HeatingEnergy(kBtu)':'L_HeatingEnergy(kBtu)',
 'CoolingEnergy(kBtu)':'L_CoolingEnergy(kBtu)',
 'LightingEnergy(kBtu)':'L_LightingEnergy(kBtu)',
 'AnnualEnergy(kBtu)':'L_AnnualEnergy(kBtu)',
 'FanEnergy(kBtu/sf)':'L_FanEnergy(kBtu/sf)',
 'HeatingEnergy(kBtu/sf)':'L_HeatingEnergy(kBtu/sf)',
 'CoolingEnergy(kBtu/sf)':'L_CoolingEnergy(kBtu/sf)',
 'LightingEnergy(kBtu/sf)':'L_LightingEnergy(kBtu/sf)',
 'EUI(kBtu/sf)':'L_EUI(kBtu/sf)',
 'CO2_Electric(lb/sf)':'L_CO2_Electric(lb/sf)',
 'CO2_Gas(lb/sf)':'L_CO2_Gas(lb/sf)',
 'FanCost($)':'L_FanCost($)',
 'HeatingCost($)':'L_HeatingCost($)',
 'CoolingCost($)':'L_CoolingCost($)',
 'LightingCost($)':'L_LightingCost($)',
 'AnnualCost($)':'L_AnnualCost($)',
 'FanCost($/sf)':'L_FanCost($/sf)',
 'HeatingCost($/sf)':'L_HeatingCost($/sf)',
 'CoolingCost($/sf)':'L_CoolingCost($/sf)',
 'LightingCost($/sf)':'L_LightingCost($/sf)',
 'AnnualCost($/sf)':'L_AnnualCost($/sf)'
                            })

# allLoweAnnualDataComElec

In [48]:
#Reset index
allLoweAnnualData.reset_index(inplace=True, drop=True)

# allLoweAnnualData

---
## Merge the 3 dataframes to perform delta from existing calculation


In [49]:
# allBaselineAnnualData
# allClearAnnualData

In [50]:
# merge existing and clear
MergedAnnualData = pd.merge(allBaselineAnnualData, allClearAnnualData, left_index=True, right_index=True)

# merge the above with lowe
AllMergedAnnualData = pd.merge(MergedAnnualData, allLoweAnnualData, left_index=True, right_index=True)

# AllMergedAnnualData

In [51]:
# list(AllMergedAnnualData.columns.values)

In [52]:
# Deltas of exist - clear

AllMergedAnnualData["DeltaClear_FanEnergy(kBtu/sf)"] = AllMergedAnnualData["B_FanEnergy(kBtu/sf)"] - AllMergedAnnualData["C_FanEnergy(kBtu/sf)"]
AllMergedAnnualData["DeltaClear_HeatingEnergy(kBtu/sf)"] = AllMergedAnnualData["B_HeatingEnergy(kBtu/sf)"] - AllMergedAnnualData["C_HeatingEnergy(kBtu/sf)"]
AllMergedAnnualData["DeltaClear_CoolingEnergy(kBtu/sf)"] = AllMergedAnnualData["B_CoolingEnergy(kBtu/sf)"] - AllMergedAnnualData["C_CoolingEnergy(kBtu/sf)"]
AllMergedAnnualData["DeltaClear_LightingEnergy(kBtu/sf)"] = AllMergedAnnualData["B_LightingEnergy(kBtu/sf)"] - AllMergedAnnualData["C_LightingEnergy(kBtu/sf)"]
AllMergedAnnualData["DeltaClear_EUI(kBtu/sf)"] = AllMergedAnnualData["B_EUI(kBtu/sf)"] - AllMergedAnnualData["C_EUI(kBtu/sf)"]
AllMergedAnnualData["DeltaClear_CO2_Electric(lb/sf)"] = AllMergedAnnualData["B_CO2_Electric(lb/sf)"] - AllMergedAnnualData["C_CO2_Electric(lb/sf)"]
# AllMergedAnnualData["DeltaClear_CO2_Gas(lb/sf)"] = AllMergedAnnualData["B_CO2_Gas(lb/sf)"] - AllMergedAnnualData["C_CO2_Gas(lb/sf)"]
AllMergedAnnualData["DeltaClear_FanCost($)"] = AllMergedAnnualData["B_FanCost($)"] - AllMergedAnnualData["C_FanCost($)"]
AllMergedAnnualData["DeltaClear_HeatingCost($)"] = AllMergedAnnualData["B_HeatingCost($)"] - AllMergedAnnualData["C_HeatingCost($)"]
AllMergedAnnualData["DeltaClear_CoolingCost($)"] = AllMergedAnnualData["B_CoolingCost($)"] - AllMergedAnnualData["C_CoolingCost($)"]
AllMergedAnnualData["DeltaClear_LightingCost($)"] = AllMergedAnnualData["B_LightingCost($)"] - AllMergedAnnualData["C_LightingCost($)"]
AllMergedAnnualData["DeltaClear_AnnualCost($)"] = AllMergedAnnualData["B_AnnualCost($)"] - AllMergedAnnualData["C_AnnualCost($)"]
AllMergedAnnualData["DeltaClear_FanCost($/sf)"] = AllMergedAnnualData["B_FanCost($/sf)"] - AllMergedAnnualData["C_FanCost($/sf)"]
AllMergedAnnualData["DeltaClear_HeatingCost($/sf)"] = AllMergedAnnualData["B_HeatingCost($/sf)"] - AllMergedAnnualData["C_HeatingCost($/sf)"]
AllMergedAnnualData["DeltaClear_CoolingCost($/sf)"] = AllMergedAnnualData["B_CoolingCost($/sf)"] - AllMergedAnnualData["C_CoolingCost($/sf)"]
AllMergedAnnualData["DeltaClear_LightingCost($/sf)"] = AllMergedAnnualData["B_LightingCost($/sf)"] - AllMergedAnnualData["C_LightingCost($/sf)"]
AllMergedAnnualData["DeltaClear_AnnualCost($/sf)"] = AllMergedAnnualData["B_AnnualCost($/sf)"] - AllMergedAnnualData["C_AnnualCost($/sf)"]


In [53]:
# AllMergedAnnualData

In [54]:
# list(AllMergedAnnualData.columns.values)

In [55]:
# Deltas of exist - lowe

AllMergedAnnualData["DeltaLowe_FanEnergy(kBtu/sf)"] = AllMergedAnnualData["B_FanEnergy(kBtu/sf)"] - AllMergedAnnualData["L_FanEnergy(kBtu/sf)"]
AllMergedAnnualData["DeltaLowe_HeatingEnergy(kBtu/sf)"] = AllMergedAnnualData["B_HeatingEnergy(kBtu/sf)"] - AllMergedAnnualData["L_HeatingEnergy(kBtu/sf)"]
AllMergedAnnualData["DeltaLowe_CoolingEnergy(kBtu/sf)"] = AllMergedAnnualData["B_CoolingEnergy(kBtu/sf)"] - AllMergedAnnualData["L_CoolingEnergy(kBtu/sf)"]
AllMergedAnnualData["DeltaLowe_LightingEnergy(kBtu/sf)"] = AllMergedAnnualData["B_LightingEnergy(kBtu/sf)"] - AllMergedAnnualData["L_LightingEnergy(kBtu/sf)"]
AllMergedAnnualData["DeltaLowe_EUI(kBtu/sf)"] = AllMergedAnnualData["B_EUI(kBtu/sf)"] - AllMergedAnnualData["L_EUI(kBtu/sf)"]
AllMergedAnnualData["DeltaLowe_CO2_Electric(lb/sf)"] = AllMergedAnnualData["B_CO2_Electric(lb/sf)"] - AllMergedAnnualData["L_CO2_Electric(lb/sf)"]
# AllMergedAnnualData["DeltaLowe_CO2_Gas(lb/sf)"] = AllMergedAnnualData["B_CO2_Gas(lb/sf)"] - AllMergedAnnualData["L_CO2_Gas(lb/sf)"]
AllMergedAnnualData["DeltaLowe_FanCost($)"] = AllMergedAnnualData["B_FanCost($)"] - AllMergedAnnualData["L_FanCost($)"]
AllMergedAnnualData["DeltaLowe_HeatingCost($)"] = AllMergedAnnualData["B_HeatingCost($)"] - AllMergedAnnualData["L_HeatingCost($)"]
AllMergedAnnualData["DeltaLowe_CoolingCost($)"] = AllMergedAnnualData["B_CoolingCost($)"] - AllMergedAnnualData["L_CoolingCost($)"]
AllMergedAnnualData["DeltaLowe_LightingCost($)"] = AllMergedAnnualData["B_LightingCost($)"] - AllMergedAnnualData["L_LightingCost($)"]
AllMergedAnnualData["DeltaLowe_AnnualCost($)"] = AllMergedAnnualData["B_AnnualCost($)"] - AllMergedAnnualData["L_AnnualCost($)"]
AllMergedAnnualData["DeltaLowe_FanCost($/sf)"] = AllMergedAnnualData["B_FanCost($/sf)"] - AllMergedAnnualData["L_FanCost($/sf)"]
AllMergedAnnualData["DeltaLowe_HeatingCost($/sf)"] = AllMergedAnnualData["B_HeatingCost($/sf)"] - AllMergedAnnualData["L_HeatingCost($/sf)"]
AllMergedAnnualData["DeltaLowe_CoolingCost($/sf)"] = AllMergedAnnualData["B_CoolingCost($/sf)"] - AllMergedAnnualData["L_CoolingCost($/sf)"]
AllMergedAnnualData["DeltaLowe_LightingCost($/sf)"] = AllMergedAnnualData["B_LightingCost($/sf)"] - AllMergedAnnualData["L_LightingCost($/sf)"]
AllMergedAnnualData["DeltaLowe_AnnualCost($/sf)"] = AllMergedAnnualData["B_AnnualCost($/sf)"] - AllMergedAnnualData["L_AnnualCost($/sf)"]


In [56]:
# list(AllMergedAnnualData.columns.values)

In [57]:
# AllMergedAnnualData

---
## Group and Export Annual Delta Data
Group dataframe by scenario<br>
Export to file

In [58]:
# Group by scenario
AnnualDataDelta = AllMergedAnnualData.groupby(["B_Scenario", "B_Elevation", "B_GlazingSystem"], as_index=False)

# AnnualDataDelta.sum()

In [59]:
# list(AnnualDataDelta.sum().columns.values)

In [60]:
AnnualDataDelta = AnnualDataDelta.sum()

In [61]:
# Export grouped file to csv
AnnualDataDelta.to_csv("Scraper_Output/riverwalk_deltas_annual.csv", header=True, index=False)

## Find Peak Demand
Slice data to find peak demand for each scenario<br>
Export to csv

In [62]:
# Copy dataframe to modify and leave original df intact
get_peak = all_energy2.copy(deep=True)

# get_peak

In [63]:
# list(get_peak.columns.values)

In [64]:
# Add energy totals -- no gas if all electric
# divide by the number of seconds in an hour (3600)
totalPeakElec = (get_peak["FanEnergy[J](Hourly)"]  + get_peak["CoolingEnergy[J](Hourly)"] + get_peak["LightingEnergy[J](Hourly)"] + get_peak["HeatingEnergy[J](Hourly)"])
# totalPeakGas =  get_peak["HeatingEnergy[J](Hourly)"]
        
get_peak["PeakElecEnergy[J](Hourly)"] = totalPeakElec
# get_peak["PeakGasEnergy[J](Hourly)"] = totalPeakGas


get_peak["PeakElecEnergy(W)"] = (get_peak["PeakElecEnergy[J](Hourly)"] / 3600)
get_peak["PeakElecEnergy(W/sf)"] = (get_peak["PeakElecEnergy(W)"] / sf)


# get_peak["PeakGasEnergy(W)"] = (get_peak["PeakGasEnergy[J](Hourly)"] / 3600)
# get_peak["PeakGasEnergy(W/sf)"] = (get_peak["PeakGasEnergy(W)"] / sf)

get_peak

,Scenario,Date_Time,Elevation,GlazingSystem,LightingEnergy[J](Hourly),CoolingEnergy[J](Hourly),HeatingEnergy[J](Hourly),FanEnergy[J](Hourly),PeakElecEnergy[J](Hourly),PeakElecEnergy(W),PeakElecEnergy(W/sf)
0,e-singletint-clear,01/01 01:00:00,east,clear panel,35099.99643,0.00000,0.00000,376064.5314,411164.52783,114.212369,0.609133
1,e-singletint-clear,01/01 02:00:00,east,clear panel,35099.99643,0.00000,0.00000,376064.5314,411164.52783,114.212369,0.609133
2,e-singletint-clear,01/01 03:00:00,east,clear panel,14849.99849,0.00000,0.00000,376064.5314,390914.52989,108.587369,0.579133
3,e-singletint-clear,01/01 04:00:00,east,clear panel,14849.99849,0.00000,0.00000,376064.5314,390914.52989,108.587369,0.579133
4,e-singletint-clear,01/01 05:00:00,east,clear panel,14849.99849,0.00000,0.00000,376064.5314,390914.52989,108.587369,0.579133
...,...,...,...,...,...,...,...,...,...,...,...
105115,w-singletint,12/31 20:00:00,west,existing,134999.98630,390794.79650,0.00000,328144.2073,853938.99010,237.205275,1.265095
105116,w-singletint,12/31 21:00:00,west,existing,134999.98630,83539.15572,0.00000,328144.2073,546683.34932,151.856486,0.809901
105117,w-singletint,12/31 22:00:00,west,existing,134999.98630,0.00000,23793.38049,328144.2073,486937.57409,135.260437,0.721389
105118,w-singletint,12/31 23:00:00,west,existing,114749.98830,0.00000,48645.53830,328144.2073,491539.73390,136.538815,0.728207


In [65]:
# get_peak["PeakGasEnergy[J](Hourly)"].idxmax()

In [66]:
# get_peak["PeakElecEnergy[J](Hourly)"].idxmax()

In [67]:
# Get scenario
Eexist = get_peak[get_peak["Scenario"] == "e-singletint"]
Eclear = get_peak[get_peak["Scenario"] == "e-singletint-clear"]
Elowe = get_peak[get_peak["Scenario"] == "e-singletint-lowe"]
Nexist = get_peak[get_peak["Scenario"] == "n-singletint"]
Nclear = get_peak[get_peak["Scenario"] == "n-singletint-clear"]
Nlowe = get_peak[get_peak["Scenario"] == "n-singletint-lowe"]
Sexist = get_peak[get_peak["Scenario"] == "s-singletint"]
Sclear = get_peak[get_peak["Scenario"] == "s-singletint-clear"]
Slowe = get_peak[get_peak["Scenario"] == "s-singletint-lowe"]
Wexist = get_peak[get_peak["Scenario"] == "w-singletint"]
Wclear = get_peak[get_peak["Scenario"] == "w-singletint-clear"]
Wlowe = get_peak[get_peak["Scenario"] == "w-singletint-lowe"]

# Wclear

In [68]:
EexistPeakGas = pd.DataFrame(Eexist.loc[Eexist['HeatingEnergy[J](Hourly)'].idxmax()])
EexistPeakGas = EexistPeakGas.swapaxes("index", "columns").reset_index().drop(['index'], axis=1)
EclearPeaKGas = pd.DataFrame(Eclear.loc[Eclear['HeatingEnergy[J](Hourly)'].idxmax()])
EclearPeaKGas = EclearPeaKGas.swapaxes("index", "columns").reset_index().drop(['index'], axis=1)
ElowePeakGas = pd.DataFrame(Elowe.loc[Elowe['HeatingEnergy[J](Hourly)'].idxmax()])
ElowePeakGas = ElowePeakGas.swapaxes("index", "columns").reset_index().drop(['index'], axis=1)
NexistPeakGas = pd.DataFrame(Nexist.loc[Nexist['HeatingEnergy[J](Hourly)'].idxmax()])
NexistPeakGas = NexistPeakGas.swapaxes("index", "columns").reset_index().drop(['index'], axis=1)
NclearPeakGas = pd.DataFrame(Nclear.loc[Nclear['HeatingEnergy[J](Hourly)'].idxmax()])
NclearPeakGas = NclearPeakGas.swapaxes("index", "columns").reset_index().drop(['index'], axis=1)
NlowePeakGas = pd.DataFrame(Nlowe.loc[Nlowe['HeatingEnergy[J](Hourly)'].idxmax()])
NlowePeakGas = NlowePeakGas.swapaxes("index", "columns").reset_index().drop(['index'], axis=1)
SexistPeakGas = pd.DataFrame(Sexist.loc[Sexist['HeatingEnergy[J](Hourly)'].idxmax()])
SexistPeakGas = SexistPeakGas.swapaxes("index", "columns").reset_index().drop(['index'], axis=1)
SclearPeakGas = pd.DataFrame(Sclear.loc[Sclear['HeatingEnergy[J](Hourly)'].idxmax()])
SclearPeakGas =  SclearPeakGas.swapaxes("index", "columns").reset_index().drop(['index'], axis=1)
SlowePeakGas = pd.DataFrame(Slowe.loc[Slowe['HeatingEnergy[J](Hourly)'].idxmax()])
SlowePeakGas = SlowePeakGas.swapaxes("index", "columns").reset_index().drop(['index'], axis=1)
WexistPeakGas = pd.DataFrame(Wexist.loc[Wexist['HeatingEnergy[J](Hourly)'].idxmax()])
WexistPeakGas = WexistPeakGas.swapaxes("index", "columns").reset_index().drop(['index'], axis=1)
WclearPeakGas = pd.DataFrame(Wclear.loc[Wclear['HeatingEnergy[J](Hourly)'].idxmax()])
WclearPeakGas =  WclearPeakGas.swapaxes("index", "columns").reset_index().drop(['index'], axis=1)
WlowePeakGas = pd.DataFrame(Wlowe.loc[Wlowe['HeatingEnergy[J](Hourly)'].idxmax()])
WlowePeakGas = WlowePeakGas.swapaxes("index", "columns").reset_index().drop(['index'], axis=1)


In [69]:
EexistPeakElec = pd.DataFrame(Eexist.loc[Eexist['CoolingEnergy[J](Hourly)'].idxmax()])
EexistPeakElec = EexistPeakElec.swapaxes("index", "columns").reset_index().drop(['index'], axis=1)
EclearPeaKElec = pd.DataFrame(Eclear.loc[Eclear['CoolingEnergy[J](Hourly)'].idxmax()])
EclearPeaKElec = EclearPeaKElec.swapaxes("index", "columns").reset_index().drop(['index'], axis=1)
ElowePeakElec = pd.DataFrame(Elowe.loc[Elowe['CoolingEnergy[J](Hourly)'].idxmax()])
ElowePeakElec = ElowePeakElec.swapaxes("index", "columns").reset_index().drop(['index'], axis=1)
# NexistPeakElec = pd.DataFrame(Nexist.loc[Nexist['CoolingEnergy[J](Hourly)'].idxmax()])
# NexistPeakElec = NexistPeakElec.swapaxes("index", "columns").reset_index().drop(['index'], axis=1)
# NclearPeakElec = pd.DataFrame(Nclear.loc[Nclear['CoolingEnergy[J](Hourly)'].idxmax()])
# NclearPeakElec = NclearPeakElec.swapaxes("index", "columns").reset_index().drop(['index'], axis=1)
# NlowePeakElec = pd.DataFrame(Nlowe.loc[Nlowe['CoolingEnergy[J](Hourly)'].idxmax()])
# NlowePeakElec = NlowePeakElec.swapaxes("index", "columns").reset_index().drop(['index'], axis=1)
# SexistPeakElec = pd.DataFrame(Sexist.loc[Sexist['CoolingEnergy[J](Hourly)'].idxmax()])
# SexistPeakElec = SexistPeakElec.swapaxes("index", "columns").reset_index().drop(['index'], axis=1)
# SclearPeakElec = pd.DataFrame(Sclear.loc[Sclear['CoolingEnergy[J](Hourly)'].idxmax()])
# SclearPeakElec =  SclearPeakElec.swapaxes("index", "columns").reset_index().drop(['index'], axis=1)
# SlowePeakElec = pd.DataFrame(Slowe.loc[Slowe['CoolingEnergy[J](Hourly)'].idxmax()])
# SlowePeakElec = SlowePeakElec.swapaxes("index", "columns").reset_index().drop(['index'], axis=1)
WexistPeakElec = pd.DataFrame(Wexist.loc[Wexist['CoolingEnergy[J](Hourly)'].idxmax()])
WexistPeakElec = WexistPeakElec.swapaxes("index", "columns").reset_index().drop(['index'], axis=1)
WclearPeakElec = pd.DataFrame(Wclear.loc[Wclear['CoolingEnergy[J](Hourly)'].idxmax()])
WclearPeakElec =  WclearPeakElec.swapaxes("index", "columns").reset_index().drop(['index'], axis=1)
WlowePeakElec = pd.DataFrame(Wlowe.loc[Wlowe['CoolingEnergy[J](Hourly)'].idxmax()])
WlowePeakElec = WlowePeakElec.swapaxes("index", "columns").reset_index().drop(['index'], axis=1)


In [70]:
PeakDemandElec = EexistPeakElec.append([EclearPeaKElec,
ElowePeakElec,
# NexistPeakElec,
# NclearPeakElec,
# NlowePeakElec,
# SexistPeakElec,
# SclearPeakElec,
# SlowePeakElec,
WexistPeakElec,
WclearPeakElec,
WlowePeakElec
])

# PeakDemandElec

In [71]:
# PeakDemandGas = EexistPeakGas.append([EclearPeaKGas,
# ElowePeakGas,
# NexistPeakGas,
# NclearPeakGas,
# NlowePeakGas,
# SexistPeakGas,
# SclearPeakGas,
# SlowePeakGas,
# WexistPeakGas,
# WclearPeakGas,
# WlowePeakGas
# ])

# PeakDemandGas

In [72]:
# # Export dataframe  to csv
# PeakDemandElec.to_csv("Scraper_Output/msp2_peakelec_com.csv", header=True, index=False)
# PeakDemandGas.to_csv("Scraper_Output/msp2_peakgas_com.csv", header=True, index=False)

---

## Establish dataframes for each condition to determine delta from existing

Create dataset with just existing condition.<br> 
Created a dataset with clear panel. <br>
Create a dataset with lowe panel.<br>
Export to CSV files


In [73]:
# Define the scenario for the existing condition.
baselinePeakDemandElec = PeakDemandElec[PeakDemandElec["GlazingSystem"] == "existing"]

# baselinePeakDemandElec

In [74]:
# list(baselinePeakDemandElec.columns.values)

In [75]:
# Rename columns to indicate baseline to prepare for merge"

allbaselinePeakDemandElec = baselinePeakDemandElec.rename(columns={'Scenario':'B_Scenario',
 'Date_Time':'B_Date_Time',
 'Elevation':'B_Elevation',
 'GlazingSystem':'B_GlazingSystem',
 'HeatingEnergy[J](Hourly)':'B_HeatingEnergy[J](Hourly)',
 'FanEnergy[J](Hourly)':'B_FanEnergy[J](Hourly)',
 'CoolingEnergy[J](Hourly)':'B_CoolingEnergy[J](Hourly)',
 'LightingEnergy[J](Hourly)':'B_LightingEnergy[J](Hourly)',
 'PeakElecEnergy[J](Hourly)':'B_PeakElecEnergy[J](Hourly)',
#  'PeakGasEnergy[J](Hourly)':'B_PeakGasEnergy[J](Hourly)',
 'PeakElecEnergy(W)':'B_PeakElecEnergy(W)',
 'PeakElecEnergy(W/sf)':'B_PeakElecEnergy(W/sf)'
# 'PeakGasEnergy(W)':'B_PeakGasEnergy(W)',
#  'PeakGasEnergy(W/sf)':'B_PeakGasEnergy(W/sf)'

                            })

allbaselinePeakDemandElec

,B_Scenario,B_Date_Time,B_Elevation,B_GlazingSystem,B_LightingEnergy[J](Hourly),B_CoolingEnergy[J](Hourly),B_HeatingEnergy[J](Hourly),B_FanEnergy[J](Hourly),B_PeakElecEnergy[J](Hourly),B_PeakElecEnergy(W),B_PeakElecEnergy(W/sf)
0,e-singletint,06/25 10:00:00,east,existing,75600,6.24566e+06,0,351288,6.67254e+06,1853.48,9.88525
0,w-singletint,05/26 19:00:00,west,existing,90450,6.0102e+06,0,328144,6.42879e+06,1785.78,9.52414


In [76]:
#Reset index
allbaselinePeakDemandElec.reset_index(inplace=True, drop=True)

allbaselinePeakDemandElec

,B_Scenario,B_Date_Time,B_Elevation,B_GlazingSystem,B_LightingEnergy[J](Hourly),B_CoolingEnergy[J](Hourly),B_HeatingEnergy[J](Hourly),B_FanEnergy[J](Hourly),B_PeakElecEnergy[J](Hourly),B_PeakElecEnergy(W),B_PeakElecEnergy(W/sf)
0,e-singletint,06/25 10:00:00,east,existing,75600,6.24566e+06,0,351288,6.67254e+06,1853.48,9.88525
1,w-singletint,05/26 19:00:00,west,existing,90450,6.0102e+06,0,328144,6.42879e+06,1785.78,9.52414


In [77]:
# Define the scenario for the clear panel condition.
clearPeakDemandElec = PeakDemandElec[PeakDemandElec["GlazingSystem"] == "clear panel"]

clearPeakDemandElec

,Scenario,Date_Time,Elevation,GlazingSystem,LightingEnergy[J](Hourly),CoolingEnergy[J](Hourly),HeatingEnergy[J](Hourly),FanEnergy[J](Hourly),PeakElecEnergy[J](Hourly),PeakElecEnergy(W),PeakElecEnergy(W/sf)
0,e-singletint-clear,06/25 08:00:00,east,clear panel,55350,6.36549e+06,0,376065,6.7969e+06,1888.03,10.0695
0,w-singletint-clear,04/23 18:00:00,west,clear panel,44550,5.81292e+06,0,335349,6.19282e+06,1720.23,9.17454


In [78]:
# Rename columns to indicate baseline to prepare for merge"

allClearPeakDemandElec = clearPeakDemandElec.rename(columns={'Scenario':'C_Scenario',
 'Date_Time':'C_Date_Time',
 'Elevation':'C_Elevation',
 'GlazingSystem':'C_GlazingSystem',
 'HeatingEnergy[J](Hourly)':'C_HeatingEnergy[J](Hourly)',
 'FanEnergy[J](Hourly)':'C_FanEnergy[J](Hourly)',
 'CoolingEnergy[J](Hourly)':'C_CoolingEnergy[J](Hourly)',
 'LightingEnergy[J](Hourly)':'C_LightingEnergy[J](Hourly)',
 'PeakElecEnergy[J](Hourly)':'C_PeakElecEnergy[J](Hourly)',
#   'PeakGasEnergy[J](Hourly)':'C_PeakGasEnergy[J](Hourly)',
'PeakElecEnergy(W)':'C_PeakElecEnergy(W)',
 'PeakElecEnergy(W/sf)':'C_PeakElecEnergy(W/sf)'
# 'PeakGasEnergy(W)':'C_PeakGasEnergy(W)',
#  'PeakGasEnergy(W/sf)':'C_PeakGasEnergy(W/sf)'
})

# allClearPeakDemandElec

In [79]:
#Reset index
allClearPeakDemandElec.reset_index(inplace=True, drop=True)

# allClearPeakDemandElec

In [80]:
# Define the scenario for the lowe panel condition.
lowePeakDemandElec = PeakDemandElec[PeakDemandElec["GlazingSystem"] == "lowe panel"]

lowePeakDemandElec

,Scenario,Date_Time,Elevation,GlazingSystem,LightingEnergy[J](Hourly),CoolingEnergy[J](Hourly),HeatingEnergy[J](Hourly),FanEnergy[J](Hourly),PeakElecEnergy[J](Hourly),PeakElecEnergy(W),PeakElecEnergy(W/sf)
0,e-singletint-lowe,08/12 08:00:00,east,lowe panel,55350,4.66943e+06,0,265657,4.99044e+06,1386.23,7.39324
0,w-singletint-lowe,05/26 19:00:00,west,lowe panel,90450,4.35061e+06,0,250181,4.69124e+06,1303.12,6.94998


In [81]:
# Rename columns to indicate baseline to prepare for merge"

allLowePeakDemandElec = lowePeakDemandElec.rename(columns={'Scenario':'L_Scenario',
 'Date_Time':'L_Date_Time',
 'Elevation':'L_Elevation',
 'GlazingSystem':'L_GlazingSystem',
 'HeatingEnergy[J](Hourly)':'L_HeatingGasEnergy[J](Hourly)',
 'FanEnergy[J](Hourly)':'L_FanEnergy[J](Hourly)',
 'CoolingEnergy[J](Hourly)':'L_CoolingEnergy[J](Hourly)',
 'LightingEnergy[J](Hourly)':'L_LightingEnergy[J](Hourly)',
 'PeakElecEnergy[J](Hourly)':'L_PeakElecEnergy[J](Hourly)',
#  'PeakGasEnergy[J](Hourly)':'L_PeakGasEnergy[J](Hourly)',
 'PeakElecEnergy(W)':'L_PeakElecEnergy(W)',
 'PeakElecEnergy(W/sf)':'L_PeakElecEnergy(W/sf)'
#  'PeakGasEnergy(W)':'L_PeakGasEnergy(W)',
#  'PeakGasEnergy(W/sf)':'L_PeakGasEnergy(W/sf)'
                            })

# allLowePeakDemandElec

In [82]:
# Reset index
allLowePeakDemandElec.reset_index(inplace=True, drop=True)

# allLowePeakDemandElec

---
## Merge the 3 dataframes to perform delta peak from existing calculation


In [83]:
# merge existing and clear
MergedAnnualPeakElecData1 = pd.merge(allbaselinePeakDemandElec, allClearPeakDemandElec, left_index=True, right_index=True)

# MergedAnnualPeakElecData1

In [84]:
# # #merge the above with lowe
MergedAnnualPeakElecData2 = pd.merge(MergedAnnualPeakElecData1, allLowePeakDemandElec, left_index=True, right_index=True)

# MergedAnnualPeakElecData2

In [85]:
# # Jpin two dataframes in B
# PeakFinal = pd.merge(MergedAnnualPeakElecData1, MergedAnnualPeakElecData2,  how="inner")

# PeakFinal

In [86]:
# list(MergedAnnualPeakElecData2.columns.values)

In [87]:
#Deltas of exist - clear

MergedAnnualPeakElecData2["DeltaClear_PeakElecEnergy(W)"] = MergedAnnualPeakElecData2["B_PeakElecEnergy(W)"] - MergedAnnualPeakElecData2["C_PeakElecEnergy(W)"]
MergedAnnualPeakElecData2["DeltaClear_PeakElecEnergy(W/sf)"] = MergedAnnualPeakElecData2["B_PeakElecEnergy(W/sf)"] - MergedAnnualPeakElecData2["C_PeakElecEnergy(W/sf)"]

# MergedAnnualPeakElecData2


In [88]:
# Deltas of exist - lowe

MergedAnnualPeakElecData2["DeltaLowe_PeakElecEnergy(W)"] = MergedAnnualPeakElecData2["B_PeakElecEnergy(W)"] - MergedAnnualPeakElecData2["L_PeakElecEnergy(W)"]
MergedAnnualPeakElecData2["DeltaLowe_PeakElecEnergy(W/sf)"] = MergedAnnualPeakElecData2["B_PeakElecEnergy(W/sf)"] - MergedAnnualPeakElecData2["L_PeakElecEnergy(W/sf)"]

# MergedAnnualPeakElecData2

In [89]:
# list(MergedAnnualPeakElecData2.columns.values)

In [90]:
# Export grouped file to csv
MergedAnnualPeakElecData2.to_csv("Scraper_Output/riverwalk_deltas_peakElec.csv", header=True, index=False)